**Exercício 1** (0,75)

Modifique o código acima para fazer com que dois processos filhos alterem uma memória compartilhada contendo um inteiro. Faça o processo pai esperar por seus dois filhos para depois imprimir o valor da variável na tela. Dica: olhe o código do exercício 3 do Laboratório 1.

---

In [1]:
%%writefile exercicio1.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/shm.h>
#include <sys/stat.h>
#include <stdlib.h>

int main(int argc, char **argv)
{
  // Criando o segmento de memória compartilhada
  int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);
  
  // Criação do ponteiro de memória compartilhada
  int *sharedMemory;

  // Criando o primeiro processo filho
  pid_t pid1 = fork();

  if (pid1 < 0) { // Erro na criação do processo
    printf("Sou o processo %d e nao consegui criar um novo processo.\n", getpid());
    return 1;
  } else if (pid1 == 0) { // PRIMEIRO PROCESSO FILHO
    // Anexando a memória compartilhada.
    sharedMemory = (int *)shmat(segment_id, NULL, 0);

    // Inicializando o valor na memória compartilhada
    *sharedMemory = 10;
    
    printf("Filho 1 (ID: %d) definiu o valor para: %d\n", getpid(), *sharedMemory);

    // Desanexando a memória compartilhada
    shmdt(sharedMemory);

    // Encerrando o processo filho
    exit(0);
  } else { // PROCESSO PAI
    // O pai cria o SEGUNDO processo filho
    pid_t pid2 = fork();

    if (pid2 < 0) { // Erro na criação do segundo processo
      printf("Sou o processo %d e nao consegui criar um segundo processo.\n", getpid());
      return 1;
    } else if (pid2 == 0) { // SEGUNDO PROCESSO FILHO
      // Anexando a memória compartilhada.
      sharedMemory = (int *)shmat(segment_id, NULL, 0);

      // O segundo filho modifica o valor já existente.
      // Ele adiciona 5 ao valor que o primeiro filho colocou (10).
      *sharedMemory += 5;

      printf("Filho 2 (ID: %d) somou 5. Novo valor: %d\n", getpid(), *sharedMemory);

      // Desanexando a memória compartilhada
      shmdt(sharedMemory);

      // Encerrando o processo filho
      exit(0);
    } else { // PROCESSO PAI NOVAMENTE
      wait(NULL);
      wait(NULL);

      // Anexando a memória compartilhada para ler o valor final.
      sharedMemory = (int *)shmat(segment_id, NULL, 0);

      // O printf agora exibe o valor final corretamente.
      printf("O valor final na memoria compartilhada é: %d\n", *sharedMemory);
      
      // Desanexando a memória
      shmdt(sharedMemory);

      // Removendo o segmento de memória compartilhada
      shmctl(segment_id, IPC_RMID, NULL);
    }
  }
  return 0;
}


Writing exercicio1.c


In [2]:
!gcc ./exercicio1.c -o exercicio1
!./exercicio1

Filho 1 (ID: 4001) definiu o valor para: 10
Filho 2 (ID: 4002) somou 5. Novo valor: 15
O valor final na memoria compartilhada é: 15


**Exercício 2** (0,75)

Utilizando comunicação entre processos, crie um código que escreva a Sequência de Fibonacci, com o parâmetro n sendo uma variável global. Tal sequência deve ser gerada por um processo filho, mas deve ser impressa na tela pelo processo pai. Dica: olhe o código do desafio do Laboratório 1.


---

In [3]:
%%writefile exercicio2.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <sys/shm.h>
#include <sys/stat.h>

int n = 20; // quantidade de termos da sequência de Fibonacci

int main(int argc, char **argv)
{
    int segment_id = shmget(IPC_PRIVATE, n * sizeof(int), S_IRUSR | S_IWUSR);

    // Criar o ponteiro para a memória compartilhada.
    int *shared_fib_array;

    pid_t pid = fork();

    if (pid == -1) {
        perror("Erro no fork");
        shmctl(segment_id, IPC_RMID, NULL); // Limpa a memória
        exit(1);
    }

    if (pid == 0) {
        // Processo filho - gera Fibonacci e escreve na memória compartilhada
        
        // Anexar a memória compartilhada ao espaço do processo filho.
        shared_fib_array = (int *)shmat(segment_id, NULL, 0);
        if (shared_fib_array == (int *)-1) {
            perror("Erro ao anexar a memória compartilhada no filho");
            exit(1);
        }

        // Gera a sequência de Fibonacci e a armazena na memória compartilhada
        shared_fib_array[0] = 0;
        if (n > 1) {
            shared_fib_array[1] = 1;
            for (int i = 2; i < n; i++) {
                shared_fib_array[i] = shared_fib_array[i-1] + shared_fib_array[i-2];
            }
        }

        // Desanexar a memória. O filho termina sua parte.
        shmdt(shared_fib_array);
        exit(0);
    } else {
        // Processo pai - espera e depois lê da memória compartilhada
        
        // O pai espera o filho terminar a tarefa antes de continuar.
        wait(NULL);

        // Anexar a memória compartilhada para que o pai possa ler.
        shared_fib_array = (int *)shmat(segment_id, NULL, 0);
        if (shared_fib_array == (int *)-1) {
            perror("Erro ao anexar a memória compartilhada no pai");
            exit(1);
        }

        // Lê e imprime a sequência da memória compartilhada
        printf("Sequência de Fibonacci até n=%d (lida da memória compartilhada):\n", n);
        for (int i = 0; i < n; i++) {
            printf("%d ", shared_fib_array[i]);
        }
        printf("\n");

        // Desanexar a memória e, por fim, a remover.
        shmdt(shared_fib_array);
        shmctl(segment_id, IPC_RMID, NULL);
    }

    return 0;
}

Writing exercicio2.c


In [4]:
!gcc ./exercicio2.c -o exercicio2
!./exercicio2

Sequência de Fibonacci até n=20 (lida da memória compartilhada):
0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 2584 4181 


**Exercício 3** (1,00)

Usando comunicação entre processos, escreva um código que realiza a soma de *n* números naturais, sendo *n* uma variável global. Cada processo deve fazer apenas a soma de dois números e o resultado final deve ser retornado pelo primeiro processo. Por exemplo, o processo 1 deve fazer a soma de 0+1, o processo 2 vai pegar esse resultado e somar com 2, e assim por diante.


---


In [ ]:
%%writefile exercicio3.c
#include <sys/types.h>
#include <sys/wait.h>
#include <stdio.h>
#include <unistd.h>
#include <stdlib.h>
#include <sys/shm.h>
#include <sys/stat.h>

int n = 10; // Variável global

int main(int argc, char **argv)
{
    // Criação do segmento de memória compartilhada para um inteiro
    int segment_id = shmget(IPC_PRIVATE, sizeof(int), S_IRUSR | S_IWUSR);
    if (segment_id == -1) {
        perror("Erro ao criar o segmento de memória compartilhada");
        exit(1);
    }
    
    int *shared_soma;
    
    // Anexa a memória compartilhada para inicializar o valor
    shared_soma = (int *)shmat(segment_id, NULL, 0);
    if (shared_soma == (int *)-1) {
        perror("Erro ao anexar a memória compartilhada");
        shmctl(segment_id, IPC_RMID, NULL);
        exit(1);
    }
    
    // Inicializa a soma com 0
    *shared_soma = 0;
    
    // Desanexa o pai temporariamente
    shmdt(shared_soma);

    // Loop para criar n processos
    for (int i = 1; i <= n; i++) {
        pid_t pid = fork();

        if (pid == -1) {
            perror("Erro no fork");
            shmctl(segment_id, IPC_RMID, NULL);
            exit(1);
        }

        if (pid == 0) { // Processo filho
            shared_soma = (int *)shmat(segment_id, NULL, 0);
            if (shared_soma == (int *)-1) {
                perror("Erro ao anexar a memória compartilhada no filho");
                exit(1);
            }
            
            // Soma o número atual ao valor acumulado
            *shared_soma += i;

            // Desanexa a memória e termina
            shmdt(shared_soma);
            exit(0);
        } else { // Processo pai
            // Espera o filho terminar de fazer a soma antes de continuar
            wait(NULL);
        }
    }
    
    // O pai finaliza o processo lendo o valor final
    shared_soma = (int *)shmat(segment_id, NULL, 0);
    if (shared_soma == (int *)-1) {
        perror("Erro ao anexar a memória compartilhada no pai");
        shmctl(segment_id, IPC_RMID, NULL);
        exit(1);
    }
    
    printf("Soma de 1 até %d = %d\n", n, *shared_soma);

    // Desanexa e remove o segmento de memória compartilhada
    shmdt(shared_soma);
    shmctl(segment_id, IPC_RMID, NULL);
    
    return 0;
}

Overwriting exercicio3.c


In [8]:
!gcc ./exercicio3.c -o exercicio3
!./exercicio3

Soma de 1 até 10 = 55
